# Multi-Armed Bandit in Web Optimization

**What?** Compare different versions of something to figure out which one is better, can be used as an alternative to AB testing.

**Why?** To *mitigate the regret* introduced in AB test, the regret about the lost conversion you experience while sending people to the potentially worse variation in a test. Rather than splitting people to two groups and run a test, the algorithm updates the split through the experiment, pulling the winning slot machine arm most often, i.e. it moves traffic towards the winning variation gradually.

**How?** Starts by sending traffic to two (or more) pages: the original and the variation, then, in an attempt to *pull the winning slot machine arm most often*, the algorithm updates based on whether or not a variation is ‘winning'. (There is also the randomization part though - the randomization of for example 10% - these trials would be used to explore the options, no matter the winning arm. It is a trade-off between trying new things in hopes of something better, and sticking with what it knows will work.)


## Multi Armed Bandit Algorithm?

- Example of reinforcement task
- Explore/exploit trade-off task in sequential decision problems
- Many algorithms available for this task
 - Thompson Sampling, one of the earliest ones actually, it is a Bayesian algorithm: The basic idea is to choose an arm to play according to its probability of being the true best arm. 


Given a set of actions with some unknown reward distributions which get better understood with the time in the experiment, the agent needs to allocate between competing choices for long-term reward maximization.

In order to find the optimal solution, we need to explore all the actions to gain additional information, but at the same time, exploit the best action to maximize the expected reward, all while having limited resources.

  

- At each step there are k bandits, we select one by an action $a \in {a_1,..a_k}$
- Each bandit has a reward $r$ associated with it - 0,1 in our case
- The reward for each action $r_a = P(r \ | \ a) $ is unknown
- The reward distribution with unknown parameters is to be learned sequentially
- Meaning, when played, each arm has $\theta_k$ probability of reward 1, each $\theta_k$ can be interpreted as an action’s success probability or mean reward. The mean rewards $\theta = \theta_1,..\theta_k$ are unknown, but fixed over time. 
- Pulling the arm produces a reward r which is sampled from the underlying distribution for that bandit
- The goal is to choose action at each time $a_t$ to maximize the reward $\sum_t r_{a_i,t}$, where $r_{a_i,t} \sim P_{a_i}$
- The arm rewards are stochastic, therefore we concentrate on maximising the expected total reward $ E(\sum_t r_{a_i,t})$
- I.e., actually, the goal is to minimize the regret
- What is the optimal arm? Different techniques.
- Easy way out would be to select each bandit one by one at a uniform rate, keep track of the rewards, then go back to the one that paid out the most (i.e. you are assessing the underlying probability distribution), but each time you spend experimenting takes you away from maximizing your reward. You are probably getting less than if you would go with the current optimal lever.
- Or second, keep pulling the bandit that gave you the max reward - here you are missing on collecting the information about the other arms that is valuable in finding the best overall arm in the long term.
- this trade-off creates the **explore-exploit dilemma**, there are different possible solutions, such as
 - *The Epsilon-greedy agent* assigns $(1- \epsilon)$ to exploit the current ‘best’ bandit, and $\epsilon$: towards exploring better bandits
 - *Thomspon Sampling* randomly allocates observations to arms according the Bayesian posterior probability that each arm is optimal, it assumes a simple prior distribution on the parameters of the reward distribution of every arm, and at any time step, play an arm according to its posterior probability of being the best arm.

## In practice
Multi-armed bandit has several approximate heuristic solutions.

[Google](https://support.google.com/analytics/answer/2846882?hl=en&topic=2844866&rd=1) uses a heuristic known as Thompson Sampling, or Randomized Probability Matching, because it combines many of the best features of these heuristics. 

Let's use Thompson Sampling then.

## Thompson Sampling

In TS, Bayes' theorem is used to update the probability for a hypothesis as more information becomes available, it derives the **posterior probability as a consequence of two: a prior probability and a "likelihood function"**.

Because we are dealing with *Click-Through Rate*, the **likelihood will be binomial distribution** - binary output, number of clicks (success) out of page views.

In Bayesian inference, for the Bernoulli and  Binomial distributions, the beta distribution is the conjugate prior probability distribution. I.e., the beta distribution can be used in Bayesian analysis to describe initial knowledge concerning probability of click - **the beta distribution is a suitable model for the random behavior of percentages and proportions.**

The initial prior will be Beta(1,1) which corresponds to uniform distribution, i.e. we do not assume anything - all probabilities of winning are equally likely, i.e. we are using minimally informative prior.

To calculate the posterior, given all acquired current data - take random sample from each bandit with its current Beta(a, b), for the bandit with the largest sample, pull it's arm, let x be the result of pulling an arm (either 0 or 1), update that bandit with the data from the last pull.

When the prior is a Beta distribution and the likelihood is a Bernoulli distribution, the posterior is also a Beta distribution. Therefore the updated posterior parameters are: a' = a + x, b' = b + 1 – x and the updated distribution is Beta(a + x, b + 1 – x)

**What do we need to keep track of?**
- number of bandits (arms)
- number of actions taken
- rewards received for each arm
- probability distribution for each arm
- the probability that each variation beats the original - stop experiment if 95% sure that a variation beats the original.
- the "potential value remaining in the experiment"- At any point in the experiment there is a winning arm, the "value remaining" in an experiment is the amount of increased conversion rate you could get by switching away from the currently winning unit.



In [186]:
import numpy as np
from scipy.stats import beta #prior/posterior distribution
from scipy.stats import binom #likelihood
from scipy.stats import bernoulli #generating testing samples (click/no click)
import datetime #set the running time of the experiment
from scipy.stats import itemfreq #MC simulation of arm probabilities


class Bandit_TS(object):
    def __init__(self, n_arm=3):
        self.n_arm =  n_arm #number of arms
        self.arm_distr = np.array([[1 for x in range(2)] for y in range(n_arm)]) #prior beta(1,1) for each arm
        self.count = np.array([0] * n_arm) #step count for each arm - number of views
        self.reward = np.array([0] * n_arm) #payoff for each arm - number of clicks
        self.k = 0 #step count
    
    
    def evaluate(self):
        """
        Update final state of beta distributions
        Output the optimal arm
        Compute optimal arm probabilities by simulation
        
        """
        for i in range(self.n_arm):
            self.arm_distr[self.n_arm-1] = np.array([1 + self.reward[self.n_arm-1], 1 + 1*self.count[self.n_arm-1] - self.reward[self.n_arm-1]])
        

    def bandit(self, stop_alpha=0.05, stop_value=0.95, iterat=1000):
        """
        Run bandit
        Stop criterion: 
        1st Use Bayes' theorem to compute the probability that variation beats others, if 95% sure that a variation beats the others then "a winner has been found"
        2nd Potential value remaining in the experiment - the "value remaining" is the amount of increased conversion rate we could get by switching away from the current champion
           To run the experiment only until we are sure that switching arms won’t help much.
           If 100% sure that an arm is optimal = 0% 'value remaining' , 
           If  70% sure that an arm is optimal = 30% 'value remaining',i.e. 30% chance that another arm is better
         End the experiment when there’s at least a 95% probability that the value remaining in the experiment is less than 1% of the champion’s conversion rate. 
         That’s a 1% improvement - if the best arm has a conversion rate of 4%, the experiment ends if the value remaining in the experiment is less than .04 percentage points of CvR.
        """   
        #nextDay = datetime.datetime.now() + timedelta(days=14) #normaly the stop criterion would be days
        
        #1st - PROBABILITY THAT VARIATIONS BEATS OTHERS
        #Let θ = (θ1, θ2,..., θk) denote the vector of conversion rates for arms 1,..k
        #Simulate a matrix containing draws of θ from the relevant beta distributions, 
        ##the rows represent random draws, 
        ##the columns represent the k arms of the experiment
        simul_m = np.zeros((10000,self.n_arm))
        stop_first = np.zeros((10000,1))
        while ((iterat!=None) and (iterat>=self.k) ):
            self.choose_arm()
            for i in range(10000):
                simul_m[i] = beta.rvs(1 + self.reward, 1 + 1*self.count - self.reward)
                stop_first[i] = np.argmax(simul_m[i])
        #MC estimate of the probability that arm a is optimal is the empirical fraction of rows for which arm a had the largest simulated value
            unique, counts = np.unique(stop_first, return_counts=True)
            arm_prob = np.array((unique, counts/10000.0),dtype='float64').T
        
        #2ND - VALUE REMAINING
        #The value remaining is the posterior distribution of (θmax-θ*)/θ*
        ##θmax = largest value of θ
        ##θ* = value of θ for the arm that is most likely to be optimal
            opt_arm = int(arm_prob[np.argmax(arm_prob[:,1], axis=0),0])
        #calculate the 95% percentile of the 'value remaining'
            stop_second = np.percentile((np.max(simul_m,axis=1) - simul_m[:,opt_arm])/ simul_m[:,opt_arm], stop_value*100)
        #End the experiment when there’s at least a 95% probability that the value remaining in the experiment is less than 1% of the champion’s conversion rate. 
        #That’s a 1% improvement - if the best arm has a conversion rate of 4%, the experiment ends if the value remaining in the experiment is less than .04 percentage points of CvR.
            if np.max(arm_prob[:,1])>=(1-stop_alpha):
                opt_arm = arm_prob[np.argmax(arm_prob[:,1], axis=0),0]
                print('The winner has been found! The arm number {} has been found as optimal at the {} confidence level after {} page views.'.format(opt_arm,stop_alpha, self.k))
                break
            elif arm_prob[np.argmax(arm_prob[:,1], axis=0),1]*0.01 >=stop_second:
                print('The winner has been found! The arm number {} has been found as optimal, as with {}% probability, the value remaining in the experiment is less than 1% possible improvement{}.'.format(opt_arm,stop_value*100,self.k))
                break   
            elif iterat==self.k:
                print('After {} iterations, the winning arm is number {}.'.format(iterat, opt_arm))    
                
        
    def choose_arm(self):
        """Choose an arm: explore x exploit"""
        #take random sample from each bandit with its current Beta(a, b) and select largest sample
        draw_arm = np.argmax(beta.rvs(1 + self.reward, 1 + 1*self.count - self.reward))
        #take the arm with largest sample and pull
        new_reward = self.pull(draw_arm)
        #update winning arm with the data from the last pull
        #When the prior is a Beta and the likelihood is a Bernoulli distribution, the posterior is Beta 
        #the updated posterior parameters are: a'=a+x,b'=b+1–x and the updated distribution is Beta(a+x,b+1–x)
        self.count[draw_arm] += 1
        self.reward[draw_arm] += new_reward
        self.k += 1
             
    def pull(self, i):
        """Pull arm and fill randomly from bernoulli distr (replace this with click results: 1-click, 0-no click)"""
        sample_bern = np.linspace(0.7,0.75,self.n_arm) #1st arm lowest probability of click, last arm highest prob. of click
        return bernoulli.rvs(sample_bern[i], size=1)

# Reference
[Multi-armed bandit experiments By Steven L. Scott, PhD, Sr. Economic Analyst, Google](https://support.google.com/analytics/answer/2844870?visit_id=1-636675832156905893-2036948797&hl=en&ref_topic=2844866&rd=1)

In [187]:
l = Bandit_TS(2)
l.count

array([0, 0])

In [188]:
l.bandit(iterat=1000)

The winner has been found! The arm number 1 has been found as optimal, as with 95.0% probability, the value remaining in the experiment is less than 1% possible improvement19.


In [151]:
print(l.count)
print(l.reward)
l.evaluate()
print(l.arm_distr)

[19 16]
[ 6 10]
[[ 1  1]
 [11  7]]


In [162]:
beta.rvs(11, 7, size=50)

array([0.72379976, 0.66726472, 0.52980372, 0.55533553, 0.63824496,
       0.76647921, 0.71634506, 0.73569613, 0.49761221, 0.64043557,
       0.72099139, 0.70033517, 0.57575517, 0.62361255, 0.56601642,
       0.49994968, 0.66068505, 0.77057935, 0.42544783, 0.63167236,
       0.43921033, 0.31596281, 0.75519304, 0.66215555, 0.64744448,
       0.69337971, 0.66021125, 0.54721379, 0.74058319, 0.53074304,
       0.63007018, 0.52745654, 0.90793326, 0.56563818, 0.4742972 ,
       0.68161565, 0.62667203, 0.69262572, 0.71669753, 0.65790517,
       0.53144114, 0.62907452, 0.52342719, 0.60086485, 0.80266375,
       0.63278772, 0.53936658, 0.5656152 , 0.7524539 , 0.71843794])

In [168]:
simul_m = np.zeros((10,2))

for i in range(10):
    simul_m[i] = beta.rvs(1 + l.reward, 1 + 1*l.count - l.reward)
    stop_first[i] = np.argmax(simul_m[i])
        #MC estimate of the probability that arm a is optimal is the empirical fraction of rows for which arm a had the largest simulated value
unique, counts = np.unique(stop_first, return_counts=True)
arm_prob = np.array((unique, counts/10000.0),dtype='float64').T

In [172]:
print(l.reward, l.count)
print(beta.rvs(1+6, 1+19-6, size=10))
print(beta.rvs(1+10, 1+16-10, size=10))

(array([ 6, 10]), array([19, 16]))
[0.19429723 0.41712598 0.50685797 0.21347753 0.38782563 0.36671591
 0.26549788 0.52436061 0.36277161 0.23188391]
[0.5699479  0.52728963 0.54156049 0.80503826 0.71875778 0.71403755
 0.71290237 0.61162877 0.79981211 0.38669764]


In [169]:
simul_m

array([[0.14441273, 0.41732086],
       [0.2405907 , 0.78256302],
       [0.19687684, 0.63634222],
       [0.31940431, 0.36277244],
       [0.33709339, 0.56462031],
       [0.26856339, 0.70051594],
       [0.4504511 , 0.58973462],
       [0.23310067, 0.58818827],
       [0.21293115, 0.54770297],
       [0.20974756, 0.38421141]])

In [170]:
stop_first

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [61]:
self_n_arm = 2
self_arm_distr = np.array([[1 for x in range(2)] for y in range(self_n_arm)]) #prior beta(1,1) for each arm
self_count = np.array([0] * self_n_arm) #step count for each arm - number of views
self_reward = np.array([0] * self_n_arm) #payoff for each arm - number of clicks
self_k = 0 #step count

simul_m = np.zeros((10,self_n_arm))
stop_first = np.zeros((10,1))


In [70]:
for i in range(10):
    simul_m[i] = beta.rvs(1 + self_reward, 1 + 1*self_count - self_reward)
    stop_first[i] = np.argmax(simul_m[i])
    #MC estimate of the probability that arm a is optimal is the empirical fraction of rows for which arm a had the largest simulated value
unique, counts = np.unique(stop_first, return_counts=True)
arm_prob = np.array((unique, counts/10.0),dtype='float64').T
opt_arm = int(arm_prob[np.argmax(arm_prob[:,1], axis=0),0])

In [71]:
print(simul_m)
print(stop_first)
print(arm_prob)
print(opt_arm)

[[0.66625673 0.93561973]
 [0.797976   0.88116723]
 [0.33837417 0.8104794 ]
 [0.38847415 0.03146294]
 [0.88990776 0.66927843]
 [0.23789105 0.47428161]
 [0.04903929 0.02032877]
 [0.7818171  0.86588795]
 [0.22002196 0.41256933]
 [0.83660258 0.5966895 ]]
[[1.]
 [1.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]
 [0.]]
[[0.  0.4]
 [1.  0.6]]
1


In [79]:
np.max(arm_prob[:,1])>=(1-stop_alpha)

False

In [147]:
print(np.linspace(0.4,0.6,2))
bernoulli.rvs(0.6, size=10)

[0.4 0.6]


array([1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

In [78]:
if np.max(arm_prob[:,1])>=(1-stop_alpha):
                opt_arm = arm_prob[np.argmax(arm_prob[:,1], axis=0),0]
                print('The winner has been found! The arm number {} has been found as optimal at the {} confidence level after {} page views.'.format(opt_arm,stop_alpha, self.k))
elif 1==1:
    print('After {} iterations, the winning arm is number {}.'.format(stop_alpha, stop_alpha)) 

After 0.05 iterations, the winning arm is number 0.05.


In [33]:


       #2ND - VALUE REMAINING
        #The value remaining is the posterior distribution of (θmax-θ*)/θ*
        ##θmax = largest value of θ
        ##θ* = value of θ for the arm that is most likely to be optimal

        #calculate the 95% percentile of the 'value remaining'
            stop_second = np.percentile((np.max(simul_m,axis=1) - simul_m[:,opt_arm])/ simul_m[:,opt_arm], stop_value*100)
        #End the experiment when there’s at least a 95% probability that the value remaining in the experiment is less than 1% of the champion’s conversion rate. 
        #That’s a 1% improvement - if the best arm has a conversion rate of 4%, the experiment ends if the value remaining in the experiment is less than .04 percentage points of CvR.
            if np.max(arm_prob[:,1])>=(1-stop_alpha):
                opt_arm = arm_prob[np.argmax(arm_prob[:,1], axis=0),0]
                print('The winner has been found! The arm number {} has been found as optimal at the {} confidence level after {} page views.'.format(opt_arm,stop_alpha, self.k))
                break
                        elif iterat==self.k:
                print('After {} iterations, the winning arm is number {}.'.format(iterat, opt_arm)) 

array([[0.02574152, 0.657901  ],
       [0.95200307, 0.8968902 ],
       [0.11924942, 0.12384715],
       [0.17294811, 0.83850458],
       [0.7093667 , 0.21201003],
       [0.61552885, 0.9916095 ],
       [0.12653478, 0.17357755],
       [0.88655027, 0.32172172],
       [0.45359271, 0.19245398],
       [0.23454614, 0.22786764]])